# Imports

In [2]:
# Imports do Selenium e WebDriver
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Exceções específicas do Selenium agrupadas
from selenium.common.exceptions import (
    TimeoutException,
    ElementNotInteractableException,
    NoSuchElementException  
)

# Bibliotecas padrão do Python
import os
import time
import urllib.request
from datetime import datetime  # Mantida a importação direta da classe 'datetime'

# Bibliotecas de terceiros
import pandas as pd
import openpyxl
import requests

# Main Code

In [3]:



XPATH_DOWNLOAD = '//*[@id="container"]/div[1]/div[3]/div/div/div/div[2]/div[3]/div[3]/div/div[3]/div/div/div[2]/div[1]/div[1]/div/div[2]/span'
XPATH_UPLOAD = '//*[@id="container"]/div[1]/div[3]/div/div/div/div[2]/div[3]/div[3]/div/div[3]/div/div/div[2]/div[1]/div[2]/div/div[2]/span'
XPATH_PING = '//*[@id="container"]/div[1]/div[3]/div/div/div/div[2]/div[3]/div[3]/div/div[3]/div/div/div[2]/div[2]/div/span[2]/span'
ID_ACEITAR = 'onetrust-accept-btn-handler'
CLASS_START_BTN = 'start-text'
XPATH_RESULTADOS_DIV = '//div[@data-result-id]'
XPATH_DOWNLOAD = '//span[@class="result-data-large number result-data-value download-speed"]'
XPATH_UPLOAD = '//span[@class="result-data-large number result-data-value upload-speed"]'
XPATH_PING = '//span[@class="result-data-value ping-speed"]'

CLASS_BOTAO_FECHAR_AVISO = 'close-btn'
class INTERNET:
    def __init__(self) -> None:
        self.base_url = "https://www.speedtest.net/pt"
        options = ChromeOptions()
        options.add_argument("--start-maximized")
        # options.add_argument("--headless") # Remova se deseja que rode em segundo plano

        try:
            service = Service(ChromeDriverManager().install())
        except ValueError:
            latest_chromedriver_version_url = "https://chromedriver.storage.googleapis.com/LATEST_RELEASE"
            latest_chromedriver_version = urllib.request.urlopen(latest_chromedriver_version_url).read().decode('utf-8')
            service = Service(ChromeDriverManager(version=latest_chromedriver_version).install())

        self.browser = Chrome(service=service, options=options)
        self.browser.get(self.base_url)


        time.sleep(3)
        WebDriverWait(self.browser, 10).until(
            EC.element_to_be_clickable((By.ID, ID_ACEITAR))
        ).click()

        WebDriverWait(self.browser, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, CLASS_START_BTN))
        ).click()
        print("Teste iniciado. Aguardando resultados...")

        time.sleep(3)
        self._fechar_aviso_se_existir()

        WebDriverWait(self.browser, 90).until(
            EC.presence_of_element_located((By.XPATH, XPATH_RESULTADOS_DIV))
        )
        print("Resultados carregados.")

        time.sleep(3)
        self._fechar_aviso_se_existir()
        self.browser.execute_script("window.scrollTo(0, 0);")

        self.extract_data()
        

    def _fechar_aviso_se_existir(self):
        """
        Verifica a presença do modal de aviso. Se encontrá-lo, espera o botão ser clicável e o fecha.
        """
        try:
            xpath_aviso_texto = '//*[contains(text(), "Voltar aos resultados do teste")]'
            WebDriverWait(self.browser, 3).until(
                EC.presence_of_element_located((By.XPATH, xpath_aviso_texto))
            )
            print("Aviso 'Voltar aos resultados do teste' detectado.")
            
            time.sleep(10)
            self.browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            # --- MODIFICAÇÃO PRINCIPAL ---
            # Espera até que o botão de fechar esteja realmente pronto para ser clicado
            print("Aguardando o botão de fechar ser interativo...")
            botao_fechar = WebDriverWait(self.browser, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="container"]/div[1]/div[3]/div/div/div/div[2]/div[2]/div/div[4]/div/div[8]/div/div/div[2]/a'))
            )
            
            botao_fechar.click()
            print("Aviso fechado com sucesso.")
            time.sleep(1)
            self.browser.execute_script("window.scrollTo(0, 0);")
            
        except TimeoutException:
            print("Nenhum aviso de aplicativo foi exibido.")
            pass
        except ElementNotInteractableException:
            print("ERRO: O botão de fechar foi encontrado mas não é interativo. Tentando clicar com JavaScript...")
            # Implementando a Solução 2 como um plano B
            try:
                botao_fechar = self.browser.find_element(By.XPATH, '//*[@id="container"]/div[1]/div[3]/div/div/div/div[2]/div[2]/div/div[4]/div/div[8]/div/div/div[2]/a')
                self.browser.execute_script("arguments[0].click();", botao_fechar)
                print("Aviso fechado com sucesso via JavaScript.")
            except Exception as e:
                print(f"Falha ao tentar clicar com JavaScript: {e}")



    def data_e_hora(self):
        """Obtém a data e hora local da máquina. Mais robusto e sem dependências externas."""
        agora = datetime.now()
        return agora.strftime('%d/%m/%Y'), agora.strftime('%H:%M:%S')

    def extract_data(self):
        """
        Extrai os dados da página e os armazena.
        """
        print("Extraindo dados do teste...")
        download = self.browser.find_element(By.XPATH, XPATH_DOWNLOAD).text
        upload = self.browser.find_element(By.XPATH, XPATH_UPLOAD).text
        ping = self.browser.find_element(By.XPATH, XPATH_PING).text

        data, hora = self.data_e_hora()

        self.info = []
        info_extraida = {
            'Data': data,
            'Hora': hora,
            'Download (Mbps)': download,
            'Upload (Mbps)': upload,
            'Ping (ms)': ping,
        }
        self.info.append(info_extraida)

        # Fecha o navegador após a extração
        self.browser.quit()

        tabela_temporaria = pd.DataFrame(self.info)
        print("Dados extraídos:")

        print(tabela_temporaria)


if __name__ == "__main__":
    internet = INTERNET()

    novos_dados = pd.DataFrame(internet.info)
    nome_arquivo = "informacoes.xlsx"

    if os.path.exists(nome_arquivo):
        print(f"Adicionando dados ao arquivo existente: {nome_arquivo}")
        df_existente = pd.read_excel(nome_arquivo)
        df_final = pd.concat([df_existente, novos_dados], ignore_index=True)
    else:
        print(f"Criando novo arquivo: {nome_arquivo}")
        df_final = novos_dados

    df_final.to_excel(nome_arquivo, index=False)
    print("Dados salvos com sucesso!")

Teste iniciado. Aguardando resultados...
Aviso 'Voltar aos resultados do teste' detectado.
Aguardando o botão de fechar ser interativo...
Nenhum aviso de aplicativo foi exibido.
Resultados carregados.
Aviso 'Voltar aos resultados do teste' detectado.
Aguardando o botão de fechar ser interativo...
Nenhum aviso de aplicativo foi exibido.
Extraindo dados do teste...
Dados extraídos:
         Data      Hora Download (Mbps) Upload (Mbps) Ping (ms)
0  29/08/2025  18:21:07          688.09             —         2
Adicionando dados ao arquivo existente: informacoes.xlsx
Dados salvos com sucesso!
